In [98]:
import pandas as pd
import nltk.data
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import gensim
from keras.models import Model
from keras.layers import Input, Embedding, Dense, Dropout, Flatten
from keras.layers import Activation
import numpy as np
import chardet
import logging
from keras.models import Model
from keras.layers import Input,Dense,Dropout,Flatten
from keras.layers import Conv1D,MaxPooling1D,AveragePooling1D
from keras.layers import LSTM,Bidirectional
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.models import load_model
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, roc_auc_score, precision_score
from sklearn.model_selection import cross_val_predict
from keras import models
from keras import layers
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification
from sklearn import cross_validation
from sklearn.model_selection import cross_validate
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from nltk.stem import PorterStemmer


In [129]:
def clean_text1(dataframe):
#Cleaning text    
    dataframe["doc"] = dataframe['text']
#Remove numbers
    dataframe.loc[:,"doc"] = dataframe.doc.str.replace(r'\d+','')
#Remove floats
    dataframe.loc[:,"doc"] = dataframe.doc.astype(str).replace(r'(\d*\.?\d*)','')
#Remove Punctuation
    dataframe.loc[:,"doc"] = dataframe.doc.astype(str).apply(lambda x : " ".join(re.findall('[\w]+',x)))
#Convert to lower text
    dataframe.loc[:,"doc"] = dataframe.doc.astype(str).apply(lambda x: " ".join(x.lower() for x in x.split()))      
#Stop words removal
    stop = stopwords.words('english')
    dataframe.loc[:,'doc'] = dataframe.loc[:,'doc'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
#Reomve top 50 common words
    freq = pd.Series(' '.join(dataframe['doc']).split()).value_counts()[:50]
    freq = list(freq.index)
    dataframe.loc[:,'doc'] = dataframe.loc[:,'doc'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
#Stemming        
    st = PorterStemmer()
    dataframe.loc[:,'doc'] = dataframe.loc[:,'doc'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))

# Moving cleaned text to another column
    dataframe['cleaned_text'] = dataframe["doc"]
#Tokenization                               
    dataframe.loc[:,"doc"] = dataframe.loc[:,"doc"].apply(nltk.word_tokenize)
    
    return dataframe

In [4]:
def w2c_model(doc,size,window,min_count,workers,iter,sg,length,numepoch):    
    model = gensim.models.Word2Vec(size=size, window=window, min_count=min_count, workers=workers,iter=iter,sg=sg)
    model.build_vocab(doc)
    model.train(doc,total_examples=length,epochs = numepoch)
    return model

In [5]:
#Convert clean text into sequence base on word2vec
def text2seq(doc, w2v, seq_len, emb_size):
    padding = np.array([0 for __ in range(emb_size)])
    seq = []
    for token in doc[: seq_len]:
        try:
            seq.append(w2v.wv[token])
        except:
            aa = 1

    seq += [padding for __ in range(seq_len - len(seq))]
    #print(len(seq))
    return seq

In [6]:
def one_hot(series):
    values=sorted(list(set(series)))
    return np.vstack(map(lambda x: [x==v for v in values],series))

In [7]:
def prepro(dataframe, w2v,seq_len, emb_size, target_col):
    X = dataframe['doc'].apply(lambda doc: text2seq(doc, w2v, seq_len, emb_size))
    X = np.vstack(X).reshape(len(dataframe), seq_len, emb_size)
    y = one_hot(dataframe[target_col])
    return X, y

In [303]:
#Model building for original approach
def model1(seq_len, emb_size, num_labels):
    input_layer = Input(shape=(seq_len,emb_size)) 
    x = LSTM(100, return_sequences = True,dropout=0.2,recurrent_dropout=0.05)(input_layer)
    x = Conv1D(64, 3, activation='relu')(x)
    x = MaxPooling1D(pool_size=2)(x) 
    x = Flatten()(x)
    output_layer = Dense(num_labels, activation='softmax')(x)
    model = Model(input_layer, output_layer)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
    return model

In [299]:
#Model for cross valiation 
def build_model3():
    input_layer = Input(shape=(100,100)) 
    x = LSTM(100, return_sequences = True,dropout=0.2,recurrent_dropout=0.05)(input_layer)
    x = Conv1D(64, 3, activation='relu')(x) #Second Change: Changed Kernal size here from 64 to 32
    x = MaxPooling1D(pool_size=2)(x) 
    x = Flatten()(x)
#     x = Dense(256, activation='relu')(x)
#     x = Dropout(0.2)(x)                   #Fourth Change: Adding dropout layer here
#     x = Dense(64, activation='relu')(x)   #Third Change: Adding dense layer here
# #     x = Dropout(0.2)(x)
#     x = Dense(128, activation='relu')(x)
#     x = Dropout(0.2)(x)
    output_layer = Dense(9, activation='softmax')(x)
    model = Model(input_layer, output_layer)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
    return model

In [304]:
def fit_model(X,y,weightsfile,num_labels,seq_len,emb_size,epochs):
    checkpoint = ModelCheckpoint(weightsfile, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    model_dl = model1(seq_len, emb_size, num_labels)
    model_dl.fit(X, y, epochs=epochs, batch_size=16,callbacks=callbacks_list, validation_split=0.2)
    return model_dl

In [307]:
def prediction(model,weightsfile,X):
    model.load_weights(weightsfile)
    preds = model.predict(X)
    lst = list(map(lambda x: x.argmax(), preds))
    return preds,lst

# Processing Data

In [11]:
df = pd.read_csv('data.csv')

In [12]:
df.head()

,Unnamed: 0,ID,text,Label
0,0,20030006121,Passive radio frequency identification system ...,8
1,1,20030009666,Methods and apparatus for efficient computatio...,0
2,2,20030014337,"Systems, methods and computer program products...",8
3,3,20030019942,System and method for electronically readable ...,8
4,4,20030033242,System and method for automated process of dea...,6


In [130]:
#Clean the text column
df = clean_text1(df)
#Assign processed column to be the target column for the word2vec
doc = df.doc
#create a word2vec model to use as embedding layers in CNN
w2c = w2c_model(doc,size=100,window=5,min_count=1,workers=6,iter=10,sg=0,length=len(doc),numepoch=10)

In [134]:
df.head()

,Unnamed: 0,ID,text,Label,doc,cleaned_text
0,0,20030006121,Passive radio frequency identification system ...,8,"[passiv, radio, frequenc, identif, identifi, t...",passiv radio frequenc identif identifi track c...
1,1,20030009666,Methods and apparatus for efficient computatio...,0,"[method, apparatu, effici, comput, way, chain,...",method apparatu effici comput way chain crypto...
2,2,20030014337,"Systems, methods and computer program products...",8,"[system, method, program, product, perform, ge...",system method program product perform gener co...
3,3,20030019942,System and method for electronically readable ...,8,"[electron, readabl, power, sourc, improv, cont...",electron readabl power sourc improv contain el...
4,4,20030033242,System and method for automated process of dea...,6,"[autom, process, deal, structur, autom, deal, ...",autom process deal structur autom deal custom ...


In [142]:
df = df.sample(frac=1)

In [175]:
#Create sequence and embedding layer base on word2vec 
X, y = prepro(df,w2c,seq_len=100, emb_size=100,target_col='Label')

In [300]:
# Wrap Keras model so it can be used by scikit-learn
CNN = KerasClassifier(build_fn=build_model3, 
                            #batch_size = 64,  #Fifth Change: batch size from 32 to 64
                            epochs=20) #First change: From 20 to 10
                                 

In [302]:
#Use cross validation predict to make prediction on the entire dataset
#The test & train accuracy is 99.12%
cnn_pred = cross_val_predict(CNN, X, y, cv=10)



Epoch 1/20
793/793 [==============================] - 71s 89ms/step - loss: 1.7771 - acc: 0.3569
Epoch 2/20
793/793 [==============================] - 3s 4ms/step - loss: 1.1210 - acc: 0.5939
Epoch 3/20
793/793 [==============================] - 3s 4ms/step - loss: 0.8191 - acc: 0.7226
Epoch 4/20
793/793 [==============================] - 3s 4ms/step - loss: 0.6302 - acc: 0.7768
Epoch 5/20
793/793 [==============================] - 3s 4ms/step - loss: 0.4847 - acc: 0.8348
Epoch 6/20
793/793 [==============================] - 3s 4ms/step - loss: 0.3735 - acc: 0.8714
Epoch 7/20
793/793 [==============================] - 3s 4ms/step - loss: 0.2640 - acc: 0.9130
Epoch 8/20
793/793 [==============================] - 3s 4ms/step - loss: 0.1952 - acc: 0.9445
Epoch 9/20
793/793 [==============================] - 3s 4ms/step - loss: 0.1660 - acc: 0.9571
Epoch 10/20
793/793 [==============================] - 3s 4ms/step - loss: 0.1373 - acc: 0.9584
Epoch 11/20
793/793 [==========================

794/794 [==============================] - 3s 4ms/step - loss: 0.1188 - acc: 0.9748
Epoch 12/20
794/794 [==============================] - 3s 4ms/step - loss: 0.0849 - acc: 0.9824
Epoch 13/20
794/794 [==============================] - 3s 4ms/step - loss: 0.0810 - acc: 0.9798
Epoch 14/20
794/794 [==============================] - 3s 4ms/step - loss: 0.0762 - acc: 0.9824
Epoch 15/20
794/794 [==============================] - 3s 4ms/step - loss: 0.0598 - acc: 0.9887
Epoch 16/20
794/794 [==============================] - 3s 4ms/step - loss: 0.0613 - acc: 0.9861
Epoch 17/20
794/794 [==============================] - 3s 4ms/step - loss: 0.0613 - acc: 0.9874
Epoch 18/20
794/794 [==============================] - 3s 4ms/step - loss: 0.0500 - acc: 0.9924
Epoch 19/20
794/794 [==============================] - 3s 4ms/step - loss: 0.0683 - acc: 0.9824
Epoch 20/20
794/794 [==============================] - 3s 4ms/step - loss: 0.0684 - acc: 0.9824
Epoch 1/20
794/794 [==============================] 

In [291]:
#Assign the prediction back to dataframe
# df['predict_b64'] = cnn_pred

In [180]:
# values=sorted(list(set(df['Label'])))
# values

[0, 1, 2, 3, 4, 5, 6, 7, 8]

In [273]:
df.head()

,Unnamed: 0,ID,text,Label,doc,cleaned_text,predict,predict1,predict2,predict3,predict_64,predict_k10,predict_e10,predict_d64,predict_de
835,835,20030187790,Electronic check processing systems The prese...,8,"[electron, check, system, includ, system, proc...",electron check system includ system process tr...,1,8,8,2,8,8,8,8,8
767,767,20110112985,METHOD AND SYSTEM FOR GENERATING A FINANCIAL P...,6,"[gener, plan, score, gener, plan, score, inclu...",gener plan score gener plan score includ plan ...,8,3,6,6,6,6,6,6,6
353,353,20150180836,CLOUD-BASED TRANSACTIONS METHODS AND SYSTEMS ...,1,"[cloud, transact, method, system, techniqu, en...",cloud transact method system techniqu enhanc s...,1,1,2,2,2,1,2,2,1
248,248,20130061305,RANDOM CHALLENGE ACTION FOR AUTHENTICATION OF ...,1,"[random, challeng, action, authent, devic, aut...",random challeng action authent devic authent e...,1,0,1,1,0,0,0,1,1
648,648,20150269538,SECURITY DEVICES AND SYSTEMS FOR DIGITAL CURRE...,4,"[secur, devic, system, digit, currenc, transfe...",secur devic system digit currenc transfer digi...,0,4,1,0,4,4,4,4,4


# Accuracy, Precision & Recall for original model

In [316]:
print(classification_report(df['Label'],df['predict_64']))

#Accuracy score
accuracy_score(df['Label'],df['predict_64'])*100

             precision    recall  f1-score   support

          0       0.75      0.80      0.77       274
          1       0.50      0.61      0.55        84
          2       0.65      0.48      0.55        77
          3       0.79      0.69      0.74        61
          4       0.66      0.57      0.61        37
          5       0.51      0.52      0.52        42
          6       0.62      0.67      0.65        45
          7       0.85      0.65      0.73        34
          8       0.63      0.64      0.64       228

avg / total       0.67      0.67      0.67       882



66.89342403628117

# Accuracy, Precision & Recall for Epochs = 10

In [317]:
print(classification_report(df['Label'],df['predict_e10']))

#Accuracy score
accuracy_score(df['Label'],df['predict_e10'])*100

             precision    recall  f1-score   support

          0       0.73      0.78      0.75       274
          1       0.66      0.49      0.56        84
          2       0.57      0.48      0.52        77
          3       0.74      0.84      0.78        61
          4       0.65      0.59      0.62        37
          5       0.56      0.45      0.50        42
          6       0.69      0.69      0.69        45
          7       0.88      0.68      0.77        34
          8       0.61      0.67      0.63       228

avg / total       0.67      0.67      0.67       882



67.00680272108843

# Accuracy, Precision & Recall for Kernal Size = 32

In [318]:
print(classification_report(df['Label'],df['predict1']))

#Accuracy score
accuracy_score(df['Label'],df['predict1'])*100

             precision    recall  f1-score   support

          0       0.73      0.74      0.73       274
          1       0.55      0.51      0.53        84
          2       0.45      0.40      0.42        77
          3       0.82      0.75      0.79        61
          4       0.72      0.57      0.64        37
          5       0.71      0.48      0.57        42
          6       0.61      0.67      0.64        45
          7       0.96      0.68      0.79        34
          8       0.57      0.68      0.62       228

avg / total       0.65      0.65      0.65       882



64.62585034013605

# Accuracy, Precision & Recall for Adding Dense Layer (64 nodes, 'relu' activation)

In [319]:
print(classification_report(df['Label'],df['predict_d64']))

#Accuracy score
accuracy_score(df['Label'],df['predict_d64'])*100

             precision    recall  f1-score   support

          0       0.72      0.76      0.74       274
          1       0.54      0.61      0.57        84
          2       0.56      0.48      0.52        77
          3       0.78      0.69      0.73        61
          4       0.73      0.59      0.66        37
          5       0.60      0.50      0.55        42
          6       0.66      0.64      0.65        45
          7       0.85      0.65      0.73        34
          8       0.60      0.65      0.62       228

avg / total       0.66      0.66      0.66       882



65.64625850340136

# Accuracy, Precision & Recall for Adding Dropout layer with dropout rate 0.2

In [323]:
print(classification_report(df['Label'],df['predict_d0.2']))

#Accuracy score
accuracy_score(df['Label'],df['predict_d0.2'])*100

             precision    recall  f1-score   support

          0       0.74      0.81      0.77       274
          1       0.58      0.64      0.61        84
          2       0.58      0.55      0.56        77
          3       0.73      0.70      0.72        61
          4       0.70      0.57      0.63        37
          5       0.62      0.55      0.58        42
          6       0.64      0.60      0.62        45
          7       0.83      0.71      0.76        34
          8       0.65      0.63      0.64       228

avg / total       0.68      0.68      0.68       882



67.91383219954649

# Accuracy, Precision & Recall for changing batch size to 64

In [324]:
print(classification_report(df['Label'],df['predict_b64']))

#Accuracy score
accuracy_score(df['Label'],df['predict_b64'])*100

             precision    recall  f1-score   support

          0       0.72      0.80      0.76       274
          1       0.54      0.54      0.54        84
          2       0.58      0.53      0.55        77
          3       0.78      0.75      0.77        61
          4       0.65      0.65      0.65        37
          5       0.57      0.38      0.46        42
          6       0.71      0.64      0.67        45
          7       1.00      0.62      0.76        34
          8       0.63      0.66      0.65       228

avg / total       0.67      0.67      0.67       882



67.00680272108843

# Training and make prediction in a traditional way

In [306]:
#Train model
#Accuracy is 99.43%
CNN_model= fit_model(X=X,y=y,weightsfile = "first.hdf5",num_labels=9,seq_len=100,emb_size=100,epochs=20)


Train on 705 samples, validate on 177 samples
Epoch 1/20
705/705 [==============================] - 64s 90ms/step - loss: 1.6177 - acc: 0.4326 - val_loss: 1.0653 - val_acc: 0.6667

Epoch 00001: val_acc improved from -inf to 0.66667, saving model to first.hdf5
Epoch 2/20
705/705 [==============================] - 5s 8ms/step - loss: 0.9571 - acc: 0.6482 - val_loss: 0.8406 - val_acc: 0.7006

Epoch 00002: val_acc improved from 0.66667 to 0.70056, saving model to first.hdf5
Epoch 3/20
705/705 [==============================] - 6s 8ms/step - loss: 0.6770 - acc: 0.7645 - val_loss: 0.9391 - val_acc: 0.6780

Epoch 00003: val_acc did not improve from 0.70056
Epoch 4/20
705/705 [==============================] - 6s 8ms/step - loss: 0.4601 - acc: 0.8355 - val_loss: 0.8874 - val_acc: 0.6554

Epoch 00004: val_acc did not improve from 0.70056
Epoch 5/20
705/705 [==============================] - 6s 8ms/step - loss: 0.3098 - acc: 0.9021 - val_loss: 0.8261 - val_acc: 0.7458

Epoch 00005: val_acc impro

In [308]:
#Make prediction on the entire dataset
preds,lst = prediction(model=CNN_model,weightsfile = "first.hdf5",X = X)

In [309]:
#Assign the prediction values back to dataframe
df['p'] = lst

In [325]:
print(classification_report(df['Label'],df['p']))
accuracy_score(df['Label'],df['p'])*100

             precision    recall  f1-score   support

          0       0.94      0.97      0.95       274
          1       0.89      0.88      0.89        84
          2       0.92      0.95      0.94        77
          3       0.92      0.97      0.94        61
          4       0.97      0.95      0.96        37
          5       0.97      0.90      0.94        42
          6       0.89      0.93      0.91        45
          7       0.91      0.94      0.93        34
          8       0.95      0.90      0.93       228

avg / total       0.93      0.93      0.93       882



93.42403628117914